# Introduction
I will develop an assistant with Huggingface. This AI assistant will use the Deepseek model and It will add comments for the Python code.

In [4]:
# Library
import os
import torch
from google.colab import drive, userdata
from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer, AutoModelForCausalLM

In [5]:
# Use the token to use the model in Huggingface
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential = True)

In [6]:
# Download and import tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code = True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code = True, torch_dtype = torch.bfloat16).cuda()

tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [7]:
# Masseges for try the model
messages=[
    { 'role': 'user', 'content': "write a quick sort algorithm in python."}
]

In [10]:
# I enter the messages with inputs. The Outputs create a message with LLM model
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Sure, here is a simple implementation of the Quick Sort algorithm in Python:

```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[0]
        less_than_pivot = [x for x in arr[1:] if x <= pivot]
        greater_than_pivot = [x for x in arr[1:] if x > pivot]
        return quick_sort(less_than_pivot) + [pivot] + quick_sort(greater_than_pivot)

# Test the function
arr = [10, 7, 8, 9, 1, 5]
print("Original array:", arr)
print("Sorted array:", quick_sort(arr))
```

This code works by selecting a 'pivot' element from the array and partitioning the other elements into two sub-arrays, according to whether they are less than or greater than the pivot. The pivot element is then in its final position. The process is then repeated for the sub-arrays.



In [12]:
# System prompt for my llm model
system_message = "You are a helpful assistant specialized in analyzing and explaining Python code. \
Your primary task is to take raw Python code as input and return the same code annotated with clear, concise, and informative \
comments in English. Your comments should explain the purpose of functions, logic of control structures, \
and the role of key variables and operations.\n"
system_message += "Write comments that would help a beginner to intermediate programmer understand the code’s structure and behavior. Keep comments aligned with the code using standard Python comment syntax (#)."
system_message += """
Input (user prompt):
def factorial(n):
  if n == 0:
    return 1
  else:
    return n * factorial(n - 1)"""

system_message += """Output (your response):
# This function calculates the factorial of a number using recursion
def factorial(n):
    # Base case: the factorial of 0 is defined as 1
    if n == 0:
        return 1
    else:
        # Recursive case: n * factorial of (n - 1)
        return n * factorial(n - 1)"""

In [13]:
system_message

'You are a helpful assistant specialized in analyzing and explaining Python code. Your primary task is to take raw Python code as input and return the same code annotated with clear, concise, and informative comments in English. Your comments should explain the purpose of functions, logic of control structures, and the role of key variables and operations.\nWrite comments that would help a beginner to intermediate programmer understand the code’s structure and behavior. Keep comments aligned with the code using standard Python comment syntax (#).\nInput (user prompt):\ndef factorial(n):\n  if n == 0:\n    return 1\n  else:\n    return n * factorial(n - 1)Output (your response):\n# This function calculates the factorial of a number using recursion\ndef factorial(n):\n    # Base case: the factorial of 0 is defined as 1\n    if n == 0:\n        return 1\n    else:\n        # Recursive case: n * factorial of (n - 1)\n        return n * factorial(n - 1)'

In [14]:
# this is a message creater for model
def createrOfMessages(python_code):
  messages = [
      {'role' : 'system', 'content' : system_message},
      {'role' : 'user', 'content' : python_code}
  ]
  return messages

In [15]:
# This function create message with my python code
def create_prompt(python_code):
  messages = createrOfMessages(python_code)
  inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
  outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
  print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

In [16]:
# Sample Python code
python_code = """
def fibonacci(n):
    a, b = 0, 1
    result = []
    for _ in range(n):
        result.append(a)
        a, b = b, a + b
    return result

def main():
    n = int(input("Kaç terim istiyorsunuz? "))
    dizi = fibonacci(n)
    print("Fibonacci dizisi:", dizi)

if __name__ == "__main__":
    main()
"""

In [17]:
create_prompt(python_code)

The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


# This function calculates the Fibonacci sequence up to the nth term
def fibonacci(n):
    # Initialize the first two terms of the sequence
    a, b = 0, 1
    # Create an empty list to store the sequence
    result = []
    # Generate the Fibonacci sequence
    for _ in range(n):
        # Append the current term to the sequence
        result.append(a)
        # Move to the next term in the sequence
        a, b = b, a + b
    # Return the generated sequence
    return result

def main():
    # Ask the user for the number of terms they want in the Fibonacci sequence
    n = int(input("Kaç terim istiyorsunuz? "))
    # Generate the Fibonacci sequence up to the nth term
    dizi = fibonacci(n)
    # Print the generated sequence
    print("Fibonacci dizisi:", dizi)

# This is the entry point of the program
if __name__ == "__main__":
    main()



# CONCLUSION
I developed an assistant and It added comments to my code. If You are learning Python programming language, this AI assistant can help you understand the code. In this project, I used the Huggingface platform and The Deepseek model. Thank you for reviewing my project. I hope it was useful for you. I will continue to share projects. If you want to be informed in advance, you can follow me from the links below.

[LinkedIn](https://www.linkedin.com/in/ihsancenkiz/)<br>
[GitHub](https://github.com/ihsncnkz)<br>
[Kaggle](https://www.kaggle.com/ihsncnkz)